In [1]:

import re
from typing import List

from src import inputs

inputs.read_input(5).split("\n")[:14]

['[J]             [F] [M]            ',
 '[Z] [F]     [G] [Q] [F]            ',
 '[G] [P]     [H] [Z] [S] [Q]        ',
 '[V] [W] [Z] [P] [D] [G] [P]        ',
 '[T] [D] [S] [Z] [N] [W] [B] [N]    ',
 '[D] [M] [R] [J] [J] [P] [V] [P] [J]',
 '[B] [R] [C] [T] [C] [V] [C] [B] [P]',
 '[N] [S] [V] [R] [T] [N] [G] [Z] [W]',
 ' 1   2   3   4   5   6   7   8   9 ',
 '',
 'move 2 from 4 to 6',
 'move 1 from 9 to 5',
 'move 3 from 2 to 4',
 'move 8 from 4 to 7']

In [2]:
world_state = inputs.read_input(5).split("\n")[:8]
world_state

['[J]             [F] [M]            ',
 '[Z] [F]     [G] [Q] [F]            ',
 '[G] [P]     [H] [Z] [S] [Q]        ',
 '[V] [W] [Z] [P] [D] [G] [P]        ',
 '[T] [D] [S] [Z] [N] [W] [B] [N]    ',
 '[D] [M] [R] [J] [J] [P] [V] [P] [J]',
 '[B] [R] [C] [T] [C] [V] [C] [B] [P]',
 '[N] [S] [V] [R] [T] [N] [G] [Z] [W]']

In [3]:
world_state[-1][1 + 8 * 4]

'W'

In [4]:
def process_line(line:str) -> List[str]:
    return_list = []
    for i in range(9):
        return_list.append(line[1 + i * 4])
        
    return return_list


In [5]:
p_row = process_line(world_state[0])
print(p_row)
len(p_row)

['J', ' ', ' ', ' ', 'F', 'M', ' ', ' ', ' ']


9

In [6]:
[process_line(p_row) for p_row in reversed(world_state)]

[['N', 'S', 'V', 'R', 'T', 'N', 'G', 'Z', 'W'],
 ['B', 'R', 'C', 'T', 'C', 'V', 'C', 'B', 'P'],
 ['D', 'M', 'R', 'J', 'J', 'P', 'V', 'P', 'J'],
 ['T', 'D', 'S', 'Z', 'N', 'W', 'B', 'N', ' '],
 ['V', 'W', 'Z', 'P', 'D', 'G', 'P', ' ', ' '],
 ['G', 'P', ' ', 'H', 'Z', 'S', 'Q', ' ', ' '],
 ['Z', 'F', ' ', 'G', 'Q', 'F', ' ', ' ', ' '],
 ['J', ' ', ' ', ' ', 'F', 'M', ' ', ' ', ' ']]

In [41]:
from pydantic import BaseModel

class Order(BaseModel):
    num_to_move: int
    start: int
    end: int

class World():
    def __init__(self, size:int, init_state: List[str]) -> None:
        self.stacks = [[] for _ in range(size)]
        
        p_state = [process_line(p_row) for p_row in reversed(world_state)]
        
        for row in p_state:
            for i, box in enumerate(row):
                if box.strip():
                    self.stacks[i].append(box)
        
    def move_stack(self, start: int, end: int) -> None:
        item = self.stacks[start].pop()
        
        self.stacks[end].append(item)

    def process_order(self, order: Order) -> None:
        for _ in range(order.num_to_move):
            self.move_stack(start=order.start - 1, end=order.end - 1)
        
    def get_top_boxes(self) -> str:
        tb = [s[-1] for s in self.stacks]
        return "".join(tb)
        

In [8]:
world = World(9, world_state)
print(world.stacks)
world.move_stack(0, 1)
world.stacks

[['N', 'B', 'D', 'T', 'V', 'G', 'Z', 'J'], ['S', 'R', 'M', 'D', 'W', 'P', 'F'], ['V', 'C', 'R', 'S', 'Z'], ['R', 'T', 'J', 'Z', 'P', 'H', 'G'], ['T', 'C', 'J', 'N', 'D', 'Z', 'Q', 'F'], ['N', 'V', 'P', 'W', 'G', 'S', 'F', 'M'], ['G', 'C', 'V', 'B', 'P', 'Q'], ['Z', 'B', 'P', 'N'], ['W', 'P', 'J']]


[['N', 'B', 'D', 'T', 'V', 'G', 'Z'],
 ['S', 'R', 'M', 'D', 'W', 'P', 'F', 'J'],
 ['V', 'C', 'R', 'S', 'Z'],
 ['R', 'T', 'J', 'Z', 'P', 'H', 'G'],
 ['T', 'C', 'J', 'N', 'D', 'Z', 'Q', 'F'],
 ['N', 'V', 'P', 'W', 'G', 'S', 'F', 'M'],
 ['G', 'C', 'V', 'B', 'P', 'Q'],
 ['Z', 'B', 'P', 'N'],
 ['W', 'P', 'J']]

In [9]:
raw_orders = inputs.read_input(5).split("\n")[10:]
raw_orders[:10]

['move 2 from 4 to 6',
 'move 1 from 9 to 5',
 'move 3 from 2 to 4',
 'move 8 from 4 to 7',
 'move 2 from 9 to 7',
 'move 3 from 8 to 3',
 'move 2 from 1 to 2',
 'move 5 from 7 to 9',
 'move 1 from 9 to 4',
 'move 1 from 8 to 3']

In [10]:
nums = re.findall(r"\d", raw_orders[0])

Order(num_to_move=nums[0], start=nums[1], end=nums[2])

Order(num_to_move=2, start=4, end=6)

In [31]:
def process_raw_order(row: str) -> Order:
    nums = re.findall(r"\d+", row)
    return Order(num_to_move=nums[0], start=nums[1], end=nums[2])

In [32]:
order_list = [process_raw_order(row) for row in raw_orders]

In [33]:
world = World(9, world_state)

print(world.stacks)

for i in range(20):
    print(order_list[i])
    world.process_order(order_list[i])

[['N', 'B', 'D', 'T', 'V', 'G', 'Z', 'J'], ['S', 'R', 'M', 'D', 'W', 'P', 'F'], ['V', 'C', 'R', 'S', 'Z'], ['R', 'T', 'J', 'Z', 'P', 'H', 'G'], ['T', 'C', 'J', 'N', 'D', 'Z', 'Q', 'F'], ['N', 'V', 'P', 'W', 'G', 'S', 'F', 'M'], ['G', 'C', 'V', 'B', 'P', 'Q'], ['Z', 'B', 'P', 'N'], ['W', 'P', 'J']]
num_to_move=2 start=4 end=6
num_to_move=1 start=9 end=5
num_to_move=3 start=2 end=4
num_to_move=8 start=4 end=7
num_to_move=2 start=9 end=7
num_to_move=3 start=8 end=3
num_to_move=2 start=1 end=2
num_to_move=5 start=7 end=9
num_to_move=1 start=9 end=4
num_to_move=1 start=8 end=3
num_to_move=1 start=3 end=4
num_to_move=2 start=4 end=9
num_to_move=7 start=3 end=5
num_to_move=6 start=1 end=8
num_to_move=11 start=7 end=9
num_to_move=12 start=5 end=3
num_to_move=6 start=6 end=9
num_to_move=3 start=3 end=8
num_to_move=4 start=2 end=7
num_to_move=3 start=5 end=7


In [38]:
world = World(9, world_state)

for o in order_list:
    world.process_order(o)

world.get_top_boxes()

'GFTNRBZPF'

# Part 2

In [48]:
class World_2():
    def __init__(self, size:int, init_state: List[str]) -> None:
        self.stacks = [[] for _ in range(size)]
        
        p_state = [process_line(p_row) for p_row in reversed(world_state)]
        
        for row in p_state:
            for i, box in enumerate(row):
                if box.strip():
                    self.stacks[i].append(box)
        
    def move_stack(self, order: Order) -> None:
        items = self.stacks[order.start - 1][-order.num_to_move:]
        
        del self.stacks[order.start - 1][-order.num_to_move:]
        
        self.stacks[order.end - 1].extend(items)

    def get_top_boxes(self) -> str:
        tb = [s[-1] for s in self.stacks]
        return "".join(tb)

In [49]:
world = World_2(9, world_state)

print(world.stacks)
print(order_list[0])
world.move_stack(order_list[0])
print(world.stacks)

[['N', 'B', 'D', 'T', 'V', 'G', 'Z', 'J'], ['S', 'R', 'M', 'D', 'W', 'P', 'F'], ['V', 'C', 'R', 'S', 'Z'], ['R', 'T', 'J', 'Z', 'P', 'H', 'G'], ['T', 'C', 'J', 'N', 'D', 'Z', 'Q', 'F'], ['N', 'V', 'P', 'W', 'G', 'S', 'F', 'M'], ['G', 'C', 'V', 'B', 'P', 'Q'], ['Z', 'B', 'P', 'N'], ['W', 'P', 'J']]
num_to_move=2 start=4 end=6
[['N', 'B', 'D', 'T', 'V', 'G', 'Z', 'J'], ['S', 'R', 'M', 'D', 'W', 'P', 'F'], ['V', 'C', 'R', 'S', 'Z'], ['R', 'T', 'J', 'Z', 'P'], ['T', 'C', 'J', 'N', 'D', 'Z', 'Q', 'F'], ['N', 'V', 'P', 'W', 'G', 'S', 'F', 'M', 'H', 'G'], ['G', 'C', 'V', 'B', 'P', 'Q'], ['Z', 'B', 'P', 'N'], ['W', 'P', 'J']]


In [50]:

world = World_2(9, world_state)

for o in order_list:
    world.move_stack(o)

world.get_top_boxes()

'VRQWPDSGP'